In [10]:
import re

from src.resolve_init import GetResolve
from pprint import pprint
from dftt_timecode import DfttTimecode

resolve = GetResolve()
project_manager = resolve.GetProjectManager()
project = project_manager.GetCurrentProject()
media_storage = resolve.GetMediaStorage()
media_pool = project.GetMediaPool()
root_folder = media_pool.GetRootFolder()
current_timeline = project.GetCurrentTimeline()

lst = ["1", "11", "10", "2", "4", "x"]


def scene_sorting_handler(scene: str):
    try:
        return int(scene)
    except ValueError:
        try:
            return int(re.split(r"[+\-&#@$*a-zA-Z\u4e00-\u9fa5]", scene)[0])
        except ValueError:
            return float("inf")


scenes = ['8', '31+1', '62', '63', '75', '77c', '88+2', '88+1', '海边空镜', '1夜空镜', '按时间看']

sorted(scenes, key=scene_sorting_handler, reverse=True)

['海边空镜', '按时间看', '88+2', '88+1', '77c', '75', '63', '62', '31+1', '8', '1夜空镜']